In [3]:
tc = [
    (["14 + 3 = 17", "13 - 6 = X", "51 - 5 = 44"], 	["13 - 6 = 5"]),
    [	["1 + 1 = 2", "1 + 3 = 4", "1 + 5 = X", "1 + 2 = X"], 	["1 + 5 = ?", "1 + 2 = 3"]],
    (	["10 - 2 = X", "30 + 31 = 101", "3 + 3 = X", "33 + 33 = X"], ["10 - 2 = 4", "3 + 3 = 10", "33 + 33 = 110"]),
    (	["2 - 1 = 1", "2 + 2 = X", "7 + 4 = X", "5 - 5 = X"], ["2 + 2 = 4", "7 + 4 = ?", "5 - 5 = 0"]),
    (["2 - 1 = 1", "2 + 2 = X", "7 + 4 = X", "8 + 4 = X"], ["2 + 2 = 4", "7 + 4 = 12", "8 + 4 = 13"])
    
]

In [22]:
def solution(expressions):
    answer = []
    
    
    def first_from_set(s):
        # set에서 첫 번째 element를 가져오는 가장 빠른 방법들 중 하나. 
        # list로 변환해서 0번째 element 가져오는 방법은 너무 느림.
        for e in s:
            break
        return e
    
    def number_str_of_base(n_base10, b):
        if n_base10 == 0:
            return "0"
        digits = []
        while n_base10:
            digits.append(int(n_base10 % b))
            n_base10 = n_base10 // b
        return "".join([str(i) for i in digits[::-1]])
    
    # 1. expression을 공백을 기준으로 parsing
    class Expr:
        def __init__ (self, expression):
            A, op, B, eq, C = expression.split()
            self.A = A
            self.op = op
            self.B = B
            self.C = C
            self.is_complete = C.isdigit()
            
        def get_numbers_set_in_expr(self):
            result = set()
            
            for n in self.A:
                result.add(int(n))
            for n in self.B:
                result.add(int(n))
            
            if self.is_complete:
                for n in self.C:
                    result.add(int(n))
            
            return result
        
        def check_available_base(self, base):
            if not self.is_complete:
                raise Exception("not available function")
            
            val_A = 0
            r_A = self.A[::-1]
            for i in range(len(self.A)):
                val_A += int(r_A[i]) * (base ** i)
                
            val_B = 0
            r_B = self.B[::-1]
            for i in range(len(self.B)):
                val_B += int(r_B[i]) * (base ** i)
                
            val_C = 0
            r_C = self.C[::-1]
            for i in range(len(self.C)):
                val_C += int(r_C[i]) * (base ** i)
                
            if self.op == "+":
                return (val_A + val_B) == val_C
            elif self.op == "-":
                return (val_A - val_B) == val_C
            else:
                raise Exception("unreachable operator")
                
        def calculate_X(self, base):
            if self.is_complete:
                raise Exception("not available function")
            
            val_A = 0
            r_A = self.A[::-1]
            for i in range(len(self.A)):
                val_A += int(r_A[i]) * (base ** i)
                
            val_B = 0
            r_B = self.B[::-1]
            for i in range(len(self.B)):
                val_B += int(r_B[i]) * (base ** i)
            
            # [중요!] 여기에서 계산된 숫자는 10진법 결과물이므로 다시 한 번 'base'에 해당하는 값으로 변환해주어야 함!
            if self.op == "+":
                s = number_str_of_base(val_A + val_B, base)
                return s
            elif self.op == "-":
                s = number_str_of_base(val_A - val_B, base)
                return s
            else:
                raise Exception("unreachable operator")
                
        def __str__(self):
            return f"{self.A} {self.op} {self.B} = {self.C}"
                
            
    
    exprs = [Expr(e) for e in expressions]
    
    
    # 2. 숫자들 := expression들로부터 숫자들만 추출
    numbers = set()
    
    for expr in exprs:
        numbers |= expr.get_numbers_set_in_expr()
    
    
    # 3. 가능한 진법 리스트 := max(2, (max(숫자들) + 1)) 진법 <= 가능한 진법 <= 9
    available_bases = range(max(2, max(numbers) + 1), 10)
    
    
    # 4-1. 완전 expression list := 완전하게 주어진 expression만 모은다.
    complete_exprs = [expr for expr in exprs if expr.is_complete]
    
    # 4-2. 모순없는 진법 리스트 := 가능한 진법 리스트를 활용하여, 완전 expression list에 적용해보고, 모순없는 진법 리스트만 추려낸다.
    good_bases = set(available_bases)
    for expr in complete_exprs:
        temp = set()
        for base in available_bases:
            if expr.check_available_base(base):
                temp.add(base)
        good_bases &= temp
    
    
    # 5. 불완전 expression list := X를 포함하는 expression만 모은다
    not_complete_exprs = [expr for expr in exprs if not expr.is_complete]
    
    # 6. 모순없는 진법을 모두 대입하여 가능한 X값들을 구하고 답이 1가지면 X를 그 답으로 replace하고, 여러 가지면 X를 ?로 replace한다.
    for expr in not_complete_exprs:
        temp = set()
        for base in good_bases:
            result = expr.calculate_X(base)
            temp.add(result)
        
        if len(temp) >= 2:
            expr.C = "?"
        elif len(temp) == 1:
            expr.C = first_from_set(temp)
        else:
            raise Exception("unreachable case")
    
    
    # 7. replace된 expression들을 모두 모아서 return한다.
    answer = [str(expr) for expr in not_complete_exprs]
    
    
    
    return answer


def expect(sol, answer):
    sol.sort()
    answer.sort()
    for i in range(len(sol)):
        if sol[i] != answer[i]:
            print(f"틀렸음! 정답: [{', '.join(sol)}], 하지만 나의 답: [{', '.join(answer)}]")
            print("\n")
            return

    print("정답!")
    print("\n")
        

for t in tc:
    expect(t[1], solution(t[0]))


정답!


정답!


정답!


정답!


정답!


